## Получение данных

In [50]:
import requests, time
from bs4 import BeautifulSoup

In [51]:
def url_processing(url): #обработка запроса
    session = requests.session()
    time.sleep(1)
    req = session.get(url)
    time.sleep(2)
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    return soup

In [52]:
def get_links(url):
    page = url_processing(url)
    links = []
    for link in page.find_all('a'):
        actualink = link.get('href')
        if "tv/news" in actualink:  
            links.append(actualink)
    return list(set(links))

### Собираем данные

Для данных возьмем новостной источник [мир24](https://mir24.tv/news/list/all). К новостным статьям прикрепелены теги, которые в целом можно считать за ключевые слова. Чтобы получить более 3000 токенов достаточно было собрать около 15 статей:

In [54]:
news_articles = get_links("https://mir24.tv/news/list/all")

In [55]:
all_texts = [] #статьи
all_tags = [] #теги к статьям
for article in news_articles[:15]:
    article_data = url_processing(article)
    article_content = article_data.find('article', {'class': 'text-format wfb'})
    full = article_content.find_all("p")
    text_data = article_content.find_all("p")[0].text
    tags = article_content.find_all("p")[-1].text.split("\n")
    tags = [tag.lower() for tag in tags[2:] if tag != ""]

    all_texts.append(text_data)
    all_tags.append("_".join(tags))

In [61]:
import pandas as pd

In [57]:
df = pd.DataFrame({"текст": all_texts, "изначальные теги": all_tags})

Запишем в файл, чтобы потом работать уже с готовыми данными

In [ ]:
writer = pd.ExcelWriter('output.xlsx')
df.to_excel(writer)
writer.save()

## Работа с данными

в раннее сохраненном документе xlsx я добавила колонку со словами, которые, на мой взгляд, можно было бы посчитать ключевыми:

In [62]:
import pandas as pd
excel_data = pd.read_excel('output.xlsx')
df = pd.DataFrame(excel_data, columns=['текст', 'изначальные теги', 'мои слова'])

In [63]:
len([word for text in df["текст"] for word in text.split()])

3072

In [64]:
df

,текст,изначальные теги,мои слова
0,В Москве сегодня настоящий «день жестянщика». ...,погода в снг,погода_снег_дождь_снегопад
1,Совет Федерации в ходе заседания одобрил закон...,детское пособие_закон_совет федерации,пособие_семья_дети
2,Международный фестиваль «Джаз Фест» начался в ...,джазовый фестиваль,фестиваль_Ереван_Армения
3,В России будет активнее развиваться система ок...,программа_гепатит с,гепатит c
4,Самым популярным паролем в 2022 году стало сло...,пароль,пароль
5,Американский президент Джо Байден оступился на...,саммит,Джо Байден_президент_саммит_лестница
6,Сегодня в России отмечают день рождения самбо....,самбо,самбо
7,Экологи объявили об открытии самой крупной из ...,океан_скат_экология,скат_популяция
8,В застоявшейся чайной заварке разрушаются все ...,здоровье_чай,чай
9,Древняя инфекция может помочь людям излечить з...,трансплантация_печень_проказа,печаень_проказа


Для эталонных ключевых слов возьмем объединение изначальных тегов и самостоятельно выделенных ключевых слов:

In [65]:
all_kwords = []
for orig_tags, my_tags in zip(df["изначальные теги"], df["мои слова"]):
    kwords = set(orig_tags.split("_")).union(set(my_tags.split("_")))
    all_kwords.append(set(kwords))

In [66]:
all_kwords

[{'дождь', 'погода', 'погода в снг', 'снег', 'снегопад'},
 {'дети', 'детское пособие', 'закон', 'пособие', 'семья', 'совет федерации'},
 {'Армения', 'Ереван', 'джазовый фестиваль', 'фестиваль'},
 {'гепатит c', 'гепатит с', 'программа'},
 {'пароль'},
 {'Джо Байден', 'лестница', 'президент', 'саммит'},
 {'самбо'},
 {'океан', 'популяция', 'скат', 'экология'},
 {'здоровье', 'чай'},
 {'печаень', 'печень', 'проказа', 'трансплантация'},
 {'вирус', 'заражение', 'инфекция', 'эксперимент'},
 {'Томск', 'инцидент', 'малыш', 'нападение собак'},
 {'внуково', 'метро', 'новая станция'},
 {'Спейси', 'суд', 'харассмент', 'шоу-бизнес'},
 {'граница', 'делимитация границы'}]

## Методы извлечения ключевых слов

### RAKE

In [68]:
# !pip install python-rake

In [73]:
import RAKE
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from string import punctuation
stop = stopwords.words('russian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [69]:
# !pip install pymorphy2

In [74]:
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize
m = MorphAnalyzer()

def get_lems(text):
    lems = [m.parse(w)[0].normal_form for w in simple_word_tokenize(text) if w not in punctuation]
    return " ".join(lems)

In [75]:
rake = RAKE.Rake(stop)

In [111]:
rake_data = []
for text in df["текст"]:
    res = rake.run(get_lems(text), maxWords=2, minFrequency=2)
    rake_data.append(res)
rake_data

[[],
 [('беременная', 1.0), ('семья', 1.0), ('ребёнок', 1.0)],
 [('армения', 1.0)],
 [('гепатит', 1.6)],
 [],
 [('лестница', 1.0), ('мероприятие', 1.0)],
 [('дзюдо', 1.0)],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

In [230]:
rake_res = []
for l in rake_data:
    new_l = [""]
    if len(l) != 0:
        mtchs = [match[0] for match in l]
        new_l = mtchs
    rake_res.append(new_l)
rake_res

[[''],
 ['беременная', 'семья', 'ребёнок'],
 ['армения'],
 ['гепатит'],
 [''],
 ['лестница', 'мероприятие'],
 ['дзюдо'],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 ['']]

С одной стороны, не очень хорошо, что столько пропусков. С другой стороны, там, где ключевые слова выделились, модель сработала хорошо

### TextRank

In [81]:
# !pip install summa

In [82]:
from summa import keywords

In [119]:
tr_data = []
for text in df["текст"]:
    tr_data.append(keywords.keywords(get_lems(text), language='russian', additional_stopwords=stop, scores=True)[:5])
tr_data

[[('район', 0.18693537722301795),
  ('страна', 0.1504647532723218),
  ('погода', 0.14329891913701576),
  ('работать', 0.1246712962479853),
  ('сегодня', 0.12319825651154094)],
 [('совет федерация', 0.7069687082546907),
  ('человек говориться', 0.008995486569678442),
  ('трудоспособный население отмечаться', 0.005464309197554165),
  ('региональный прожиточный минимум', 0.005464309197553997),
  ('выплачиваться нуждаться', 0.0036081761502664267)],
 [('джаз', 0.3536975054665852),
  ('концерт', 0.17585684758030662),
  ('фестиваль', 0.16960839916938333),
  ('формат', 0.15913956638468585),
  ('музыка', 0.14789003073231544)],
 [('необходимый', 0.16780901543738336),
  ('необходимо', 0.16780901543738336),
  ('гепатит', 0.14367714762608494),
  ('салон', 0.1432478195888589),
  ('программа', 0.14141987485645457)],
 [('комбинация', 0.25267717279718654),
  ('популярный пароль', 0.2165560254152213),
  ('слово password англ', 0.2109365148876811),
  ('замыкать', 0.1708630377431273),
  ('цифра', 0.156249

In [120]:
tr_res = []
for l in tr_data:
    new_l = []
    for match in l:
        new_l.append(match[0])
    tr_res.append(new_l)

In [121]:
tr_res

[['район', 'страна', 'погода', 'работать', 'сегодня'],
 ['совет федерация',
  'человек говориться',
  'трудоспособный население отмечаться',
  'региональный прожиточный минимум',
  'выплачиваться нуждаться'],
 ['джаз', 'концерт', 'фестиваль', 'формат', 'музыка'],
 ['необходимый', 'необходимо', 'гепатит', 'салон', 'программа'],
 ['комбинация',
  'популярный пароль',
  'слово password англ',
  'замыкать',
  'цифра'],
 ['байден', 'саммит', 'обменяться', 'автомобиль', 'индонезия джоко'],
 ['самбо', 'харлампиев', 'харлампий', 'борьба', 'вид'],
 ['популяция скат', 'эколог', 'являться', 'численность', 'регион учёный'],
 ['чай', 'диетолог', 'чайный', 'полезный', 'трусков специалист'],
 ['заболевание печень', 'орган', 'клетка', 'врач учёный', 'мочь'],
 ['новый', 'вирус', 'd', 'который', 'позволять'],
 ['собака', 'томск', 'ребёнок это', 'инцидент', 'пёс'],
 ['год', 'мэр', 'станция', 'метрополитен', 'метро'],
 ['обвинение', 'суд', 'американский актёр кевин спейся', 'пять год', 'лондон'],
 ['узбек

### TF-IDF

In [90]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [91]:
corpus = []
vectorizer = TfidfVectorizer()
for text in df["текст"]:
    corpus.append(" ".join([w for w in get_lems(text).split() if w not in stop]))
X = vectorizer.fit_transform(corpus)
feature_names = vectorizer.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [103]:
tfidf_res = [] #код нашла здесь: https://stackoverflow.com/questions/34449127/sklearn-tfidf-transformer-how-to-get-tf-idf-values-of-given-words-in-documen
for ind_text, text in enumerate(df["текст"]):
    words_data = dict()
    feature_index = X[ind_text, :].nonzero()[1]
    for w, s in [(feature_names[i], s) for (i, s) in zip(feature_index, [X[ind_text, x] for x in feature_index])]:
        words_data[w] = s
    tfidf_res.append([w for w, s in list(reversed(sorted(words_data.items(), key=lambda item: item[1])))[:5]])

In [104]:
tfidf_res

[['погода', 'дождь', 'снег', 'ветер', 'плюс'],
 ['пособие', 'семья', 'ребёнок', 'беременная', 'закон'],
 ['джаз', 'армения', 'ереван', 'джудит', 'концерт'],
 ['гепатит', 'больной', 'план', 'лечение', 'минздрав'],
 ['пароль', 'слово', 'комбинация', '2022', 'password'],
 ['байден', 'индонезия', 'президент', 'оступиться', 'лестница'],
 ['самбо', 'борьба', 'вид', 'харлампиев', 'дзюдо'],
 ['скат', 'популяция', 'эколог', 'регион', 'численность'],
 ['чай', 'диетолог', 'чайный', 'вредный', 'терапевт'],
 ['печень', 'клетка', 'орган', 'помочь', 'проказа'],
 ['вирус', 'новый', 'клетка', 'учёный', 'проникать'],
 ['томск', 'собака', 'малыш', 'ребёнок', 'инцидент'],
 ['станция', 'внуково', 'мэр', 'солнцевский', 'метрополитен'],
 ['актёр', 'спейся', 'обвинение', 'кевин', 'суд'],
 ['узбекистан', 'земля', 'соглашение', 'граница', 'водохранилище']]

## Шаблон

In [246]:
templates = [["NOUN"], ["NOUN+NOUN"], ["ADJ+NOUN"]]

In [247]:
def filter(kwords):
    filtered_kwords = []
    poses = []
    if len(kwords) > 0:
        for line in kwords:
            print(line)
            line_poses = [m.parse(w)[0].tag.POS for w in line.split() if m.parse(w)[0].tag.POS != None]
            poses.append(["+".join(line_poses)])
    for ind_w, w in enumerate(kwords):
        if poses[ind_w] in templates:
            filtered_kwords.append(w)
    return filtered_kwords

In [248]:
df_2 = pd.DataFrame({"RAKE" : rake_res,
                     "RAKE_filterd" : [filter(kw_line) for kw_line in rake_res],
                     "TextRank" : tr_res,
                     "TextRank_Filterd" : [filter(kw_line) for kw_line in tr_res],
                     "TF-IDF" : tfidf_res,
                     "TF-IDF_FILTERED" : [filter(kw_line) for kw_line in tfidf_res]})
df_2


беременная
семья
ребёнок
армения
гепатит

лестница
мероприятие
дзюдо








район
страна
погода
работать
сегодня
совет федерация
человек говориться
трудоспособный население отмечаться
региональный прожиточный минимум
выплачиваться нуждаться
джаз
концерт
фестиваль
формат
музыка
необходимый
необходимо
гепатит
салон
программа
комбинация
популярный пароль
слово password англ
замыкать
цифра
байден
саммит
обменяться
автомобиль
индонезия джоко
самбо
харлампиев
харлампий
борьба
вид
популяция скат
эколог
являться
численность
регион учёный
чай
диетолог
чайный
полезный
трусков специалист
заболевание печень
орган
клетка
врач учёный
мочь
новый
вирус
d
который
позволять
собака
томск
ребёнок это
инцидент
пёс
год
мэр
станция
метрополитен
метро
обвинение
суд
американский актёр кевин спейся
пять год
лондон
узбекистан
население
земля
граница
страна
погода
дождь
снег
ветер
плюс
пособие
семья
ребёнок
беременная
закон
джаз
армения
ереван
джудит
концерт
гепатит
больной
план
лечение
минздрав
пароль
слово
ко

,RAKE,RAKE_filterd,TextRank,TextRank_Filterd,TF-IDF,TF-IDF_FILTERED
0,[],[],"[район, страна, погода, работать, сегодня]","[район, страна, погода]","[погода, дождь, снег, ветер, плюс]","[погода, дождь, снег, ветер]"
1,"[беременная, семья, ребёнок]","[семья, ребёнок]","[совет федерация, человек говориться, трудоспо...",[совет федерация],"[пособие, семья, ребёнок, беременная, закон]","[пособие, семья, ребёнок, закон]"
2,[армения],[армения],"[джаз, концерт, фестиваль, формат, музыка]","[джаз, концерт, фестиваль, формат, музыка]","[джаз, армения, ереван, джудит, концерт]","[джаз, армения, ереван, джудит, концерт]"
3,[гепатит],[гепатит],"[необходимый, необходимо, гепатит, салон, прог...","[гепатит, салон, программа]","[гепатит, больной, план, лечение, минздрав]","[гепатит, план, лечение, минздрав]"
4,[],[],"[комбинация, популярный пароль, слово password...","[комбинация, цифра]","[пароль, слово, комбинация, 2022, password]","[пароль, слово, комбинация]"
5,"[лестница, мероприятие]","[лестница, мероприятие]","[байден, саммит, обменяться, автомобиль, индон...","[байден, саммит, автомобиль, индонезия джоко]","[байден, индонезия, президент, оступиться, лес...","[байден, индонезия, президент, лестница]"
6,[дзюдо],[дзюдо],"[самбо, харлампиев, харлампий, борьба, вид]","[самбо, харлампиев, харлампий, борьба, вид]","[самбо, борьба, вид, харлампиев, дзюдо]","[самбо, борьба, вид, харлампиев, дзюдо]"
7,[],[],"[популяция скат, эколог, являться, численность...","[популяция скат, эколог, численность, регион у...","[скат, популяция, эколог, регион, численность]","[скат, популяция, эколог, регион, численность]"
8,[],[],"[чай, диетолог, чайный, полезный, трусков спец...","[чай, диетолог, трусков специалист]","[чай, диетолог, чайный, вредный, терапевт]","[чай, диетолог, терапевт]"
9,[],[],"[заболевание печень, орган, клетка, врач учёны...","[заболевание печень, орган, клетка, врач учёны...","[печень, клетка, орган, помочь, проказа]","[печень, клетка, орган, проказа]"


## Оценка

In [220]:
import numpy as np

In [233]:
def pr_rec_f(pred, standard):
    tp = set(pred).intersection(standard) 
    fp = set(pred) - tp
    fn = set(standard) - tp
    if len(tp) == 0:
        prec = 0
        rec = 0
        f = 0
    else:
        prec = len(tp) / (len(tp) + len(fp))
        rec = len(tp) / (len(tp) + len(fn))
        f = 2 * ((prec * rec) / (prec + rec))
    return prec, rec, f

In [250]:
def total_results(pred, standard):
    precs = []
    recs = []
    fs = []
    for a, b in zip(pred, standard):
        p, r, f = pr_rec_f(a, b)
        precs.append(p)
        recs.append(r)
        fs.append(f)
    ave_prec, ave_rec, ave_f = np.mean(precs), np.mean(recs), np.mean(fs)
    return {"Precision": ave_prec, "Recall": ave_rec, "F-score": ave_f}

In [251]:
print("RAKE", total_results(df_2["RAKE"], all_kwords))
print("RAKE_filterd", total_results(df_2["RAKE_filterd"], all_kwords))
print("TextRank", total_results(df_2["TextRank"], all_kwords))
print("TextRank_Filtered", total_results(df_2["TextRank_Filterd"], all_kwords))
print("TF-IDF", total_results(df_2["TF-IDF"], all_kwords))
print("TF-IDF_FILTERED", total_results(df_2["TF-IDF_FILTERED"], all_kwords))

RAKE {'Precision': 0.05555555555555555, 'Recall': 0.027777777777777776, 'F-score': 0.03703703703703704}
RAKE_filterd {'Precision': 0.06666666666666667, 'Recall': 0.027777777777777776, 'F-score': 0.03888888888888888}
TextRank {'Precision': 0.14666666666666664, 'Recall': 0.2744444444444445, 'F-score': 0.18105820105820106}
TextRank_Filtered {'Precision': 0.24222222222222226, 'Recall': 0.2744444444444445, 'F-score': 0.21735449735449733}
TF-IDF {'Precision': 0.2800000000000001, 'Recall': 0.4622222222222222, 'F-score': 0.32371813371813374}
TF-IDF_FILTERED {'Precision': 0.34666666666666673, 'Recall': 0.4622222222222222, 'F-score': 0.3664021164021164}
